Il s'agit ici de la fonction QuickCertify qui est la premiere etape de notre methode de Test Systematique de la robustesse des modeles KNN. Ici, a partir de certaines conditions, nous verifions la robustesse. Ces conditions suffisantes sont conçues pour éviter l’étape la plus coûteuse de l’algorithme KNN, qui est la phase d’apprentissage qui s’appuie sur des validations croisées p-fold pour calculer le paramètre K optimal.

In [20]:
from sklearn.neighbors import KDTree
from collections import Counter
import numpy as np


In [21]:
def nearNeighborsSetKD(data, labels, k, input_x, y=None, n=0):
    tree = KDTree(data)  # Construit un arbre KD à partir des points
    dists, indices = tree.query([input_x], k=k+n)  # Trouve les k+n voisins les plus proches
    
    neighbors = [(data[i], labels[i]) for i in indices[0]]  # Récupère les voisins avant suppression
    
    if y is not None and n > 0:
        # Filtrer pour supprimer n éléments avec le label y parmi les k+n voisins
        neighbors_filtered = [neighbor for neighbor in neighbors if neighbor[1] != y][:k] + \
                             [neighbor for neighbor in neighbors if neighbor[1] == y][:-n]
        neighbors = neighbors_filtered[:k]

    return neighbors

In [22]:
def mostFreqLabelKD(data, labels, k, input_x, y=None, n=0):
    neighbors = nearNeighborsSetKD(data, labels, k, input_x, y, n)
    neighbor_labels = [label for _, label in neighbors]
    return Counter(neighbor_labels).most_common(1)[0][0]

In [23]:
def QuickCertifyKD(data, labels, n, input_x, y):
    label_set = []
    Kset = [1, 2, 3]  # Ensemble de valeurs de K à tester

    for k in Kset:
        # Obtenir l'étiquette la plus fréquente sans suppression
        y_initial = mostFreqLabelKD(data, labels, k, input_x)
        label_set.append(y_initial)
        # Obtenir l'étiquette la plus fréquente avec suppression de n éléments de label y
        y_after_removal = mostFreqLabelKD(data, labels, k, input_x, y, n)

        if y_initial != y_after_removal:
            return False
        
        if len(label_set) != 1:
            return False

    return True

In [25]:
# Charger l'ensemble de données Iris
X, y = load_iris(return_X_y=True)

# Exemple d'utilisation avec l'ensemble de données Iris
input_x = X[50]  # Utiliser un élément de l'ensemble Iris comme exemple
n = 5  # Nombre d'éléments à considérer comme potentiellement empoisonnés
label_y = y[50]  # Utiliser le label réel de l'input_x comme exemple de label à supprimer

# Appliquer QuickCertifyKD sur l'ensemble de données Iris
result = QuickCertifyKD(X, y, n, input_x, label_y)
result

False